In [1]:
import glob
import numpy as np
import scipy as sp
import einops as eo
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation
import magpylib as magpy
import os
import pandas as pd
from datetime import timedelta
import sys
from datetime import datetime

M0 = 1480 #mT
shape = [5*25.4/16, 25.4/8] #radius and height
x0 = np.array([0,0,10, 0,0,1])

In [2]:
def getField_cylinder(x, positions, M0, shape):
    B=magpy.getB(
        sources="Cylinder",
        position=x[:3],
        orientation=Rotation.align_vectors(x[3:], np.array([0, 0, 1]))[0],
        observers=positions,
        dimension=shape,
        polarization=(0, 0, M0),
    )
    return B

def cost_cylinder(x, B, positions, M0, shape):
    diff = getField_cylinder(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)

def B_dipole(position, rotation, M0, shape):
    R = np.sqrt(np.sum(position**2, axis=1))
    B = (M0 * (shape[0]) ** 2 * shape[1] / (16)) * (
        (
            3
            * position
            / R[:, np.newaxis] ** 5
            * (eo.einsum(position, rotation, "sensor dim,  dim -> sensor"))[
                :, np.newaxis
            ]
        )
        - rotation[np.newaxis, :] / (R[:, np.newaxis] ** 3)
    )
    return B


def getField_dipole(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def getField_dipole_fixed(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def cost_dipole(x, B, positions, M0, shape):
    diff = getField_dipole(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)


def minimize(x0, B, positions, M0, shape, *args):
    #print("Starting mimimization")
    b_args = (B, positions, M0, shape)
    cons = [{"type": "eq", "fun": lambda x: x[3] ** 2 + x[4] ** 2 + x[5] ** 2 - 1}]
    bounds = [(-100, 100), (-100, 100), (0, 100), (-1, 1), (-1, 1), (-1, 1)]
    res = sp.optimize.minimize(
        fun=cost_dipole, x0=x0, args=b_args, tol=1e-100, constraints=cons, bounds=bounds, *args
    ) 
    #print(f"Finished mimimization with shape {b_args[3]} at {res}")
    return res.x 

# Function to extract the timestamp part from the file path
def extract_timestamp(file_path):
    # Extract the filename from the file path (without the extension)
    filename = os.path.basename(file_path).replace('.npz', '')
    
    # Split the filename into date, time, and millisecond parts
    date_part = filename.split('-')[0]  # Extracting the 'YYYYMMDD' part
    time_part = filename.split('-')[1]  # Extracting the 'HHMMSS' part
    millisecond_part = filename.split('-')[2]  # Extracting the millisecond part
    
    # Combine date, time, and milliseconds into a full timestamp string
    full_datetime_str = f"{date_part} {time_part}.{millisecond_part}"
    
    # Create a datetime object
    full_datetime = datetime.strptime(full_datetime_str, "%Y%m%d %H%M%S.%f")
    
    return full_datetime

def importdata(folder):

    file_list = sorted(glob.glob(f'{folder}/*.npz'))
    #print(file_list)
    data = np.load(file_list[0])
    pos = data['pos'][0]
    B = [np.array(pos)]
    timestamps = []

    for x in range(len(file_list)):
        try:
            data = np.load(file_list[x])
        except:
            display(file_list[x])
        mags = data['mags']
        pos = data['pos'][0]
        mags = np.mean(mags, axis=0)
        
        B = np.append(B, [mags], axis=0)
        timestamps.append(extract_timestamp(file_list[x]))  
    B = np.delete(B, 0, axis=0)

    return B, timestamps, pos

def fitting(B, pos):
    locpred = [np.array([0,0,0])]
    anglepred = [np.array([0,0,0])]

    for x in range(len(B)):
        mags = B[x]
        #print(mags)
        x_res = minimize(x0, mags, pos, M0, shape)
        locpred = np.append(locpred, [x_res[0:3]], axis=0)
        #print(locpred)
        anglepred = np.append(anglepred, [x_res[3:6]], axis=0)

    locpred = np.delete(locpred, 0, axis=0)
    anglepred = np.delete(anglepred, 0, axis=0)
    

    return locpred, anglepred

def sensorplots(B, timestamps, sensor):
    mag = [np.array([0, 0, 0])]
    for x in range(len(B)):
        mag = np.append(mag, [B[x][sensor]], axis=0)
    mag = np.delete(mag, 0, axis=0)

    measuredbx = go.Scatter(x=timestamps, y=mag[:,0], mode='markers')
    layoutbx = go.Layout(title='b_x over time, sensor ' + str(sensor))
    figbx = go.Figure(data=[measuredbx], layout=layoutbx)
    figbx.update_yaxes(scaleanchor='y')
    figbx.update_layout(
        xaxis_title="Time",
        yaxis_title="b_x [mT]"
    )
    figbx.show()

    measuredby = go.Scatter(x=timestamps, y=mag[:,1], mode='markers')
    layoutby = go.Layout(title='b_y over time sensor ' + str(sensor))
    figby = go.Figure(data=[measuredby], layout=layoutby)
    figby.update_yaxes(scaleanchor='y')
    figby.update_layout(
        xaxis_title="Time",
        yaxis_title="b_y [mT]"
    )
    figby.show()

    measuredbz = go.Scatter(x=timestamps, y=mag[:,2], mode='markers')
    layoutbz = go.Layout(title='b_z over time sensor ' + str(sensor))
    figbz = go.Figure(data=[measuredbz], layout=layoutbz)
    figbz.update_yaxes(scaleanchor='y')
    figbz.update_layout(
        xaxis_title="Time",
        yaxis_title="b_z [mT]"
    )
    figbz.show()


In [3]:
def importdata_corr(folder, threshold=5):
    file_list = sorted(glob.glob(f'{folder}/*.npz'))
    data = np.load(file_list[0])
    pos = data['pos'][0]
    B = [np.array(pos)]
    timestamps = []

    # Initialize previous value to the first reading
    previous_mags = np.mean(data['mags'], axis=0)

    for x in range(len(file_list)):
        try:
            data = np.load(file_list[x])
        except:
            print(f"Could not load file: {file_list[x]}")
            continue  # Skip this file if it can't be loaded

        mags = data['mags']
        pos = data['pos'][0]
        mags = np.mean(mags, axis=0)

        # Calculate relative difference and check for a discontinuity
        relative_diff = np.abs((mags - previous_mags) / (previous_mags + 1e-8))  # Avoid division by zero
        if np.any(relative_diff > threshold) and np.any(np.sign(previous_mags) != np.sign(mags)):
            mags = -mags  # Flip the sign to maintain continuity if there's an abrupt sign change

        B = np.append(B, [mags], axis=0)
        timestamps.append(extract_timestamp(file_list[x]))

        previous_mags = mags  # Update previous value

    B = np.delete(B, 0, axis=0)

    return B, timestamps, pos

In [ ]:
folder = os.path.join('..', './magnet2')
B, timestamps, pos = importdata(folder)

In [6]:
display(B)

array([[[ 0.35169  , -0.2646   , -0.6754704],
        [ 0.19638  , -0.33207  , -0.877492 ],
        [-0.03546  , -0.35106  , -0.9054672],
        [-0.2826   , -0.27162  , -0.7733836],
        [ 0.41283  , -0.0984   , -0.764962 ],
        [ 0.23118  , -0.08052  , -1.017126 ],
        [-0.03885  , -0.13968  , -1.1038104],
        [-0.35646  , -0.0669   , -0.9157764],
        [ 0.41673  ,  0.11964  , -0.750442 ],
        [ 0.21954  ,  0.1797   , -0.9832944],
        [-0.04866  ,  0.24102  , -1.0517804],
        [-0.3012   ,  0.16974  , -0.8497588],
        [ 0.34383  ,  0.30363  , -0.5825908],
        [ 0.16956  ,  0.41289  , -0.8016008],
        [-0.06462  ,  0.42081  , -0.84821  ],
        [-0.27453  ,  0.37704  , -0.6956532]]])

In [8]:
sensorplots(B, timestamps, 2)